In [41]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts

In [42]:
from general_functions.call_api_with_account_id import call_api_with_service_token, call_api_with_accountId

In [43]:
from datetime import timezone
import pandas as pd


def transform_date_to_timestamp_milliseconds(date):
    """
    Function to transform datesting in
    timestamp with milliseconds
    Args:
        date: date string
    """
    date = pd.to_datetime(date).replace(tzinfo=timezone.utc).timestamp() * 1000
    date = int(date)
    return date

In [44]:
import os
import logging
from src.utils.call_api_with_account_id import call_api_with_service_token, call_api_with_accountId, send_to_innkeepr_api_id_based_paging
from_date = "2025-10-27"
to_date = (pd.to_datetime(from_date) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
SERVICE_TOKEN = os.environ["SERVICE_TOKEN"]
TARGETING_URL = os.environ["URL"]
accounts = call_api_with_service_token(
        f"{TARGETING_URL}/core/accounts/query", SERVICE_TOKEN, {}, logging
)
account_id =[acc["id"] for acc in accounts if acc["name"] == "Rosental"][0]
content={
     "created": {
                     "$gte": transform_date_to_timestamp_milliseconds(from_date),
                     "$lte": transform_date_to_timestamp_milliseconds(to_date),
               },
               "name":"checkout_completed"
                }
conv_resp = call_api_with_accountId(
            endpoint_url=f"{TARGETING_URL}/conversions/query",
            service_token=SERVICE_TOKEN,
            accountID=account_id,
            content=content,
            logger=logging,
        )
print(f"conversions found = {len(conv_resp)}")

In [45]:
conversions = pd.json_normalize(conv_resp)
conversions = conversions[
        conversions["signal.source"] == "5ee5dc05aeb18c0427aa3e06"
    ]
conversions["created"].min(), conversions["created"].max(), len(conversions)

In [46]:
temp = call_api_with_accountId(
            endpoint_url=f"{TARGETING_URL}/profiles/query",
            service_token=SERVICE_TOKEN,
            accountID=account_id,
            content={"anonymousId": conversions["anonymousId"].dropna().unique().tolist()},
            logger=logging,
        )
print(f"profiles found = {len(temp)}")

In [49]:
conv_profiles = pd.json_normalize(temp)
conv_profiles = conv_profiles.explode("externalIds")
conv_profiles["ext_name"] = conv_profiles["externalIds"].apply(lambda x: x.get("name") if isinstance(x, dict) else None)
conv_profiles.shape

In [48]:
conv_profiles_count = conv_profiles.drop_duplicates(subset=["anonymousId","ext_name"])["ext_name"].value_counts()
conv_profiles_count = conv_profiles_count.reset_index()
conv_profiles_count["perc"] = conv_profiles_count["count"]/len(conversions["anonymousId"].dropna().unique())*100
conv_profiles_count